In [ ]:
#conda install -c conda-forge librosa
#conda install -c conda-forge ffmpeg
import librosa
import numpy as np 
import librosa.display
import matplotlib.pyplot as plt
import pickle
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import urllib
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from os import listdir
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler, MinMaxScaler

# Functions to get features of each song and average them by album

In [ ]:
cid = "enter your cid here"
secret = "enter your client secret here"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_Album_Averages(recording_mbids):
    
    chroma_stft_ = []
    spec_cent_ = []
    spec_bw_ = []
    rolloff_ = []
    zcr_ = []
    mfcc_ = []
    
    for recording_mbid in recording_mbids:
        #print(recording_mbid)
        recording_mbid_url = 'https://musicbrainz.org/recording/' + recording_mbid
        raw_html = simple_get(recording_mbid_url)
        html = BeautifulSoup(raw_html, 'html.parser')
        
        for p in html.select('title'):
            title = str(p.text)
        #print(title)
        
        bad_chars = [';', ':', '!', "*", "?", "/", "'", "\""] 
        
        track_start = len("Recording") + 2
        track_stop = title.find("by") - 2
        trackName = title[track_start:track_stop]
        
        if "’" in trackName: 
            trackName = trackName.replace("’", "'")
            
        for i in bad_chars : 
            trackName = trackName.replace(i, '') 
            
        #print(trackName)
        
        artist_start = title.find("by") + 3
        artist_stop = title.find("-") - 1
        artistName = title[artist_start:artist_stop]

        if "’" in artistName: 
            artistName = trackName.replace("’", "'")
            
        for i in bad_chars : 
            artistName = artistName.replace(i, '') 
            
        #print(artistName)
                                          
        song_status = get_mp3(trackName, artistName)
        
        if song_status == "Song found":
            
            #print(song_status)
        
            chroma_stft, spec_cent, spec_bw, rolloff, zcr, mfcc = extract_features(trackName)

            chroma_stft_.append(chroma_stft)
            spec_cent_.append(spec_cent)
            spec_bw_.append(spec_bw)
            rolloff_.append(rolloff)
            zcr_.append(zcr)
            mfcc_.append(mfcc)
            
        else:
            print(song_status)
    
    #print("starting to average")
    if chroma_stft_ != []:
        mfcc_1 = []
        mfcc_2 = []
        mfcc_3 = []
        mfcc_4 = []
        mfcc_5 = []

        for i in mfcc_:
            mfcc_1.append(i[0])
            mfcc_2.append(i[1])
            mfcc_3.append(i[2])
            mfcc_4.append(i[3])
            mfcc_5.append(i[4])

        chroma_stft_avg = sum(chroma_stft_)/len(chroma_stft_)
        spec_cent_avg = sum(spec_cent_)/len(spec_cent_)
        spec_bw_avg = sum(spec_bw_)/len(spec_bw_)
        rolloff_avg = sum(rolloff_)/len(rolloff_)
        zcr_avg = sum(zcr_)/len(zcr_)
        mfcc_1_avg = sum(mfcc_1)/len(mfcc_1)
        mfcc_2_avg = sum(mfcc_2)/len(mfcc_2)
        mfcc_3_avg = sum(mfcc_3)/len(mfcc_3)
        mfcc_4_avg = sum(mfcc_4)/len(mfcc_4)
        mfcc_5_avg = sum(mfcc_5)/len(mfcc_5)

        album_averages = [chroma_stft_avg, spec_cent_avg, spec_bw_avg, rolloff_avg, zcr_avg, mfcc_1_avg, mfcc_2_avg, mfcc_3_avg, mfcc_4_avg, mfcc_5_avg]
        
    else:
        
        album_averages = []

    return album_averages

def get_mp3(trackName, artistName):
    search_string = trackName + " " + artistName
    results = sp.search(q= search_string, limit=10, offset=0, type='track', market=None)
    song_results = results['tracks']['items']
    
    if song_results != []:
        #print("found the song!")
        song = song_results[0]
        preview_url = song['preview_url']
        #print(preview_url)
        
        if preview_url != None:
            urllib.request.urlretrieve (preview_url, "mp3s/" + trackName + ".mp3")
            return "Song found"
        
        else: 
            return "Preview not found"
        
    else: 
        return "Song not found"
    
def extract_features(trackName):
   
    y, sr = librosa.load("mp3s/"+ trackName + ".mp3", duration = 15)
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    spec_cent = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    zcr = np.mean(librosa.feature.zero_crossing_rate(y))
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 5)
    
    mfcc_coef = []
    for c in mfcc:
        mfcc_coef.append(np.mean(c))
             
    return chroma_stft, spec_cent, spec_bw, rolloff, zcr, mfcc_coef

# Transform MuMu Dataset

In [ ]:
infile = open('dfTracks.pkl','rb')
track_dataset = pickle.load(infile)
infile.close()

In [ ]:
print(track_dataset)

**Create a dataset with amazonids, recording mbids, and genres**

In [ ]:
track_df_dataset = {"amazonIDs": track_dataset['amazon_id'], "recording_mbids": track_dataset['recording_mbid'], "genres": track_dataset['genres']}
track_df = pd.DataFrame(track_df_dataset)
track_df.head()

**Combine genres by amazonIDs**

In [ ]:
track_df1 = track_df.groupby('amazonIDs')['genres'].apply(', '.join).reset_index()
track_df1.head()

In [ ]:
track_df1.shape

**Combine recording_mbids by amazonIDs**

In [ ]:
track_df2 = track_df.groupby('amazonIDs')['recording_mbids'].apply(', '.join).reset_index()
track_df2.head()

In [ ]:
track_df2.shape #should be the same shape as track_df1

**Combine df1 and df2 to have a new dataset that groups recording mbids and genres by its amazon ids**

In [ ]:
new_dataset = {"amazonIDs": track_df1['amazonIDs'], "recording_mbids": track_df2['recording_mbids'], "genres": track_df1['genres']}
track_df = pd.DataFrame(new_dataset)
track_df.head()

In [ ]:
track_df.to_csv('whole_audio_unprocessed_dataset.csv')

# Reduce audio dataset to have same albums as text dataset

**Load audio data from step above**

In [ ]:
audio_df = pd.read_csv('whole_audio_unprocessed_dataset.csv')
audio_df = audio_df.drop(columns = ['Unnamed: 0'])

In [ ]:
audio_df

**Load preprocessed text data and match amazon ids then sort them st amazon ids are in the same order for both datasets**

In [ ]:
text_df = pd.read_pickle("df2000_babelfied.pkl")
entries_to_keep = text_df.amazonIDs.tolist()
audio_df = audio_df[audio_df['amazonIDs'].isin(entries_to_keep)]
text_df = text_df.sort_values(by=['amazonIDs'])

In [ ]:
audio_df = audio_df.reset_index()
audio_df

In [ ]:
audio_df.to_csv('audio_unprocessed_MLP.csv')

In [ ]:
text_df = text_df.reset_index()

In [ ]:
text_df

In [ ]:
text_df.to_csv('text_babelified_processed_MLP.csv')

# Preprocessing

In [ ]:
audio_df = pd.read_csv('audio_unprocessed_MLP.csv')
audio_df = audio_df.drop(columns = ['Unnamed: 0'])
audio_df

In [ ]:
chroma_stft = []
spec_cent = []
spec_bw = []
rolloff = []
zcr = []
mfcc_1 = []
mfcc_2 = []
mfcc_3 = []
mfcc_4 = []
mfcc_5 = []
useless_albums_Index = []
useless_albums_index = []
    
for row in audio_df.itertuples():
    recording_mbids = row.recording_mbids
    split_recording_mbids = recording_mbids.split(', ')
    album_averages = get_Album_Averages(split_recording_mbids)
    
    if album_averages != []:
        chroma_stft.append(album_averages[0])
        spec_cent.append(album_averages[1])
        spec_bw.append(album_averages[2])
        rolloff.append(album_averages[3])
        zcr.append(album_averages[4])
        mfcc_1.append(album_averages[5])
        mfcc_2.append(album_averages[6])
        mfcc_3.append(album_averages[7])
        mfcc_4.append(album_averages[8])
        mfcc_5.append(album_averages[9])
        
    else:
        print("useless album:", row.Index)
        useless_albums_Index.append(row.Index)
        useless_albums_index.append(row.index)

In [ ]:
print(len(chroma_stft))
print(len(spec_cent))
print(len(spec_bw))
print(len(rolloff))
print(len(zcr))
print(len(mfcc_1))
print(len(mfcc_2))
print(len(mfcc_3))
print(len(mfcc_4))
print(len(mfcc_5))

In [ ]:
print(len(useless_albums_Index))
print(useless_albums_Index)

In [ ]:
print(len(useless_albums_index))
print(useless_albums_index)

In [ ]:
dropped_audio_df = audio_df.drop(index = useless_albums_Index)
print(dropped_audio_df.shape)

In [ ]:
dropped_audio_df.head(10)

In [ ]:
dropped_audio_df = dropped_audio_df.reset_index(drop = True)

In [ ]:
dropped_audio_df.head(10)

In [ ]:
#print(track_Name)
dataset = {"index": dropped_audio_df['index'], "amazonIDs": dropped_audio_df['amazonIDs'], 
           "Chroma Stft": chroma_stft, "Spectral Centroid": spec_cent, "Spectral Bandwidth": spec_bw, 
           "Spectral Rolloff": rolloff, "ZCR": zcr, "MFCC 1": mfcc_1, "MFCC 2": mfcc_2, "MFCC 3": mfcc_3, 
           "MFCC 4": mfcc_4, "MFCC 5": mfcc_5, "Genres": dropped_audio_df['genres']}
df = pd.DataFrame(dataset)
df.head()

In [ ]:
df.to_csv('551_albums_audio.csv')